In [1]:
import cv2
import os.path
import pandas as pd

from tqdm import tqdm
from functions import getLocalSymmetry

<h3>Creating Directory and Meta Files</h3>

In [2]:
FOLDER_PATH = 'symmetry_database'
COLUMNS = ['fileName','startAxisX','startAxisY','endAxisX','endAxisY','centerX','centerY','width','height','label','initialRotation','overFlow','padding',
           'finalRotation','resizingPercent','Darkness','xPeriod','yPeriod','turbSize','offsetX','offsetY','backgroundType']
SHAPE = (224,224,3)
MNIST = pd.read_csv('MNIST/mnist_test.csv')
MNIST.index.name = 'index'

# Check if folder to store the data exist
if not os.path.isdir(FOLDER_PATH):
    os.mkdir(FOLDER_PATH)
# Check if folder for images exist
if not os.path.isdir(os.path.join(FOLDER_PATH, 'images')):
    os.mkdir(os.path.join(FOLDER_PATH, 'images'))
# Check if labels exist
if not os.path.isfile(os.path.join(FOLDER_PATH, 'labels.csv')):
    temp = pd.DataFrame(columns=COLUMNS)
    temp.index.name = 'index'
    temp.to_csv(os.path.join(FOLDER_PATH, 'labels.csv'))

<h3>Already Generated Images</h3>

In [3]:
prevData = pd.read_csv(os.path.join(FOLDER_PATH,'labels.csv'),index_col='index')
prevData

,fileName,startAxisX,startAxisY,endAxisX,endAxisY,centerX,centerY,width,height,label,...,finalRotation,resizingPercent,Darkness,xPeriod,yPeriod,turbSize,offsetX,offsetY,backgroundType,turbPower
index,,,,,,,,,,,,,,,,,,,,,
0,0.png,187.463081,92.894784,175.895960,61.114379,181.679520,77.004581,71.20,33.82,7,...,200,178,0.057465,1,9,16,-16,-90,1,1.752740
1,1.png,104.085794,178.842962,118.931736,191.300186,111.508765,185.071574,38.76,19.38,2,...,50,102,0.761230,3,3,64,0,0,0,1.246344
2,2.png,191.898172,177.515819,199.401828,189.524181,195.650000,183.520000,38.94,14.16,1,...,32,118,0.295845,0,5,32,-114,-169,1,2.531573
3,3.png,172.232064,31.370527,190.984832,15.635087,181.608448,23.502807,57.60,24.48,0,...,130,144,0.371654,6,7,64,0,0,0,0.277354
4,4.png,5.795300,203.449018,15.871956,188.509761,10.833628,195.979390,24.38,18.02,4,...,146,106,0.221848,2,7,8,0,0,0,1.450254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,5719.png,16.083408,205.883829,17.414221,216.722433,16.748815,211.303131,31.92,10.92,9,...,7,84,0.237822,5,6,16,0,0,0,0.278251
5720,5720.png,176.792706,159.533830,216.339012,170.873551,196.565859,165.203690,53.24,41.14,4,...,74,242,0.311352,1,1,32,0,0,0,1.382353
5721,5721.png,26.182760,173.519586,37.469702,198.870473,31.826231,186.195030,62.90,27.75,0,...,24,185,0.373697,9,6,64,0,0,0,1.414266


<h2>Creating Images</h2>

In [57]:
# Checking starting point for loop
start = len(prevData)

# Loop
for i in tqdm(range(start,len(MNIST))):
    # File name
    fileName = f'{i}.png'
    # Generating local symmetry
    img, dictSym, dictBack = getLocalSymmetry(SHAPE, MNIST, idx=i)
    # Removing unnecesary data
    del dictSym['startAxis']
    del dictSym['endAxis']
    del dictSym['center']
    # Merging dictionaries
    finalDict = dictSym | dictBack
    # Adding data
    finalDict['fileName'] = fileName
    finalDict['overFlow'] = int(finalDict['overFlow'] == True)
    # Appending to df and saving
    newRowDf = pd.DataFrame(finalDict, index=[0])
    prevData = pd.concat([prevData, newRowDf], ignore_index=True)
    prevData.index.name = 'index'
    prevData.to_csv(os.path.join(FOLDER_PATH, 'labels.csv'))
    # Saving image
    cv2.imwrite(os.path.join('symmetry_database','images',fileName), img)  

  0%|          | 0/9999 [00:03<?, ?it/s]
